In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_columns", None)


In [2]:
#读取数据
raw_data=pd.read_excel('raw_data.xlsx',index_col=0)
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 405 entries, 1 to 405
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   电池类型         405 non-null    object 
 1   PN结位置        405 non-null    object 
 2   IBC          405 non-null    object 
 3   双面           405 non-null    object 
 4   c-Si厚度       405 non-null    object 
 5   c-Si电阻率      405 non-null    object 
 6   c-Si晶相       405 non-null    object 
 7   c-Si提拉方式     405 non-null    object 
 8   材料           405 non-null    object 
 9   制备方式         405 non-null    object 
 10  厚度           405 non-null    object 
 11  poly-Si沉积方式  405 non-null    object 
 12  poly-Si厚度    405 non-null    object 
 13  poly-Si掺杂方式  405 non-null    object 
 14  额外掺杂         405 non-null    object 
 15  晶化温度         405 non-null    object 
 16  氢化方式         405 non-null    object 
 17  VOC          405 non-null    object 
 18  JSC          405 non-null    object 
 19  FF      

In [3]:
df=raw_data.copy()

#c-Si厚度
df['c-Si厚度']=np.where(df['c-Si厚度']=='—',np.nan,df['c-Si厚度'])
df['c-Si厚度']=df['c-Si厚度'].astype(float)

#c-Si电阻率
df['c-Si电阻率']=np.where(df['c-Si电阻率']=='—',np.nan,df['c-Si电阻率'])
df['c-Si电阻率']=df['c-Si电阻率'].astype(float)

#厚度
df['厚度']=np.where(df['厚度']=='—',np.nan,df['厚度'])
df['厚度']=df['厚度'].astype(float)

#poly-Si厚度
df['poly-Si厚度']=np.where(df['poly-Si厚度']=='—',np.nan,df['poly-Si厚度'])
df['poly-Si厚度']=df['poly-Si厚度'].astype(float)

#晶化温度
df['晶化温度']=np.where(df['晶化温度']=='—',np.nan,df['晶化温度'])
df['晶化温度']=df['晶化温度'].astype(float)

#VOC
df['VOC']=np.where(df['VOC']=='—',np.nan,df['VOC'])
df['VOC']=df['VOC'].astype(float)
#JSC
df['JSC']=np.where(df['JSC']=='—',np.nan,df['JSC'])
df['JSC']=df['JSC'].astype(float)
#FF
df['FF']=np.where(df['FF']=='—',np.nan,df['FF'])
df['FF']=df['FF'].astype(float)

df_na=df.copy()
#用平均值填充缺失值
df['c-Si厚度']=df['c-Si厚度'].fillna(df['c-Si厚度'].mean())
df['c-Si电阻率']=df['c-Si电阻率'].fillna(df['c-Si电阻率'].mean())
df['厚度']=df['厚度'].fillna(df['厚度'].mean())
df['poly-Si厚度']=df['poly-Si厚度'].fillna(df['poly-Si厚度'].mean())
df['晶化温度']=df['晶化温度'].fillna(df['晶化温度'].mean())

df['c-Si晶相']=df['c-Si晶相'].astype(str)
#删除因变量带有缺失值的样本
df=df.dropna(subset=['VOC','JSC','FF'])
#删除效率低于20的样本
#df=df[df['Efficiency']>20]


# 关联规则挖掘

In [4]:
#选择类别变量和Efficiency

cols=['电池类型','PN结位置','IBC','双面','c-Si晶相','c-Si提拉方式','材料',
               '制备方式','poly-Si沉积方式','poly-Si掺杂方式','额外掺杂','氢化方式']+['Efficiency']
df2=df[cols].copy()
#对Efficiency按照20以下低 20-24中 24以上高进行分组
df2['Efficiency']=pd.cut(df2['Efficiency'],bins=[0,20,24,27]).astype(str)
#对每列的值标注列名称
for col in df2.columns:
    df2[col]=col+'_'+df2[col]
df2.head()

,电池类型,PN结位置,IBC,双面,c-Si晶相,c-Si提拉方式,材料,制备方式,poly-Si沉积方式,poly-Si掺杂方式,额外掺杂,氢化方式,Efficiency
1,电池类型_n,PN结位置_前,IBC_否,双面_否,c-Si晶相_100,c-Si提拉方式_FZ,材料_SiOx,制备方式_HNO3,poly-Si沉积方式_—,poly-Si掺杂方式_in-situ,额外掺杂_无,氢化方式_H-rich layers,"Efficiency_(20, 24]"
2,电池类型_n,PN结位置_前,IBC_否,双面_否,c-Si晶相_100,c-Si提拉方式_FZ,材料_SiOx,制备方式_HNO3,poly-Si沉积方式_—,poly-Si掺杂方式_in-situ,额外掺杂_无,氢化方式_H-rich layers,"Efficiency_(0, 20]"
3,电池类型_n,PN结位置_前,IBC_否,双面_否,c-Si晶相_100,c-Si提拉方式_FZ,材料_SiOx,制备方式_HNO3,poly-Si沉积方式_—,poly-Si掺杂方式_in-situ,额外掺杂_无,氢化方式_H-rich layers,"Efficiency_(0, 20]"
4,电池类型_n,PN结位置_前,IBC_否,双面_否,c-Si晶相_100,c-Si提拉方式_FZ,材料_SiOx,制备方式_HNO3,poly-Si沉积方式_—,poly-Si掺杂方式_in-situ,额外掺杂_无,氢化方式_H-rich layers,"Efficiency_(20, 24]"
5,电池类型_n,PN结位置_前,IBC_否,双面_否,c-Si晶相_100,c-Si提拉方式_FZ,材料_SiOx,制备方式_HNO3,poly-Si沉积方式_—,poly-Si掺杂方式_in-situ,额外掺杂_无,氢化方式_H-rich layers,"Efficiency_(20, 24]"


In [5]:
#!pip install efficient-apriori -i https://pypi.tuna.tsinghua.edu.cn/simple
from efficient_apriori import apriori

#将数据转换为transaction类型
transactions=[]
for i in range(len(df2)):
    transactions.append(tuple(df2.iloc[i]))

In [6]:
itemsets,rules=apriori(transactions,min_support=0.2,min_confidence=0.7)

In [13]:
#将规则写入数据框
#左项
lhs=[]
#右项
rhs=[]
#支持度
support=[]
#规则的长度
rule_len=[]
#置信度
confidence=[]
#提升度
lift=[]
#规则在transactions中出现的次数
count=[]
for rule in rules:
    #找出右项包含Efficiency的规则
    if 'Efficiency' in ''.join(rule.rhs) and len(rule.rhs)==1:         
        lhs.append(rule.lhs)    
        rhs.append(rule.rhs)
        support.append(rule.support)
        rule_len.append(len(rule))
        confidence.append(rule.confidence)
        lift.append(rule.lift)  
        count.append(rule.count_full)

rules_df=pd.DataFrame({'lhs':lhs,
                       'rhs':rhs,
                       'support':support,
                       'confidence':confidence,
                       'lift':lift,
                       'count':count,
                       'rule_len':rule_len})
#对规则按照提升度从高到底进行排序
rules_df.sort_values(by='lift',ascending=False,inplace=True)
#将rules_df输出到excel文件
rules_df.to_excel('rules_df.xlsx',index=False)

In [24]:
rules_df.head(3)

,lhs,rhs,support,confidence,lift,count,rule_len
92,"(IBC_否, c-Si晶相_—, 双面_否)","(Efficiency_(20, 24],)",0.338308,0.871795,1.279057,136,4
311,"(IBC_否, c-Si晶相_—, 双面_否, 额外掺杂_无)","(Efficiency_(20, 24],)",0.315920,0.869863,1.276222,127,5
107,"(IBC_否, poly-Si沉积方式_LPCVD, 双面_否)","(Efficiency_(20, 24],)",0.328358,0.868421,1.274107,132,4


In [21]:
((df2['IBC']=='IBC_否') & (df2['c-Si晶相']=='c-Si晶相_—') & (df2['双面']=='双面_否')).sum()

156

In [22]:
((df2['IBC']=='IBC_否') & (df2['c-Si晶相']=='c-Si晶相_—') & (df2['双面']=='双面_否') &(df2['Efficiency']=='Efficiency_(20, 24]')).sum()

136

## 规则含义说明

第一条关联规则说明：
(IBC_否, c-Si晶相_—, 双面_否)->(Efficiency_(20, 24],)
X->Y，X表示规则的左项，Y表示规则的右项，
这里X是(IBC_否, c-Si晶相_—, 双面_否)，Y是(Efficiency_(20, 24],)

支持度support=0.338,就是说P(X,Y)=0.338;
置信度confidence=0.87  , P(Y|X)=0.87;
提升度lift=P(Y|X)/P(Y)=1.27;

含义为：
在IBC=否 且 c-Si晶相= — 且 双面=否时，Efficiency有87%的可能性为(20, 24]。
在IBC=否 且 c-Si晶相= — 且 双面=否时，Efficiency为(20, 24]的概率是一般情况下Efficiency为(20, 24]的概率的1.27倍。


In [15]:
rules_df.shape

(970, 7)

In [18]:
df2['Efficiency'].value_counts()

Efficiency_(20, 24]    274
Efficiency_(0, 20]      98
Efficiency_(24, 27]     30
Name: Efficiency, dtype: int64